#INSTALAR LIBRERIAS Y PROBLEMAS



In [1]:
!wget -O pybex1.9.0.tar.gz 'https://drive.google.com/uc?id=1C1SGBoC46aBhtFqVyQMIM0x_spHDEzgK'  &> /dev/null
!pip install pybex pybex1.9.0.tar.gz  &> /dev/null

In [2]:
!git clone https://github.com/juanfuentealbaR/tft_problems

Cloning into 'tft_problems'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 252 (delta 17), reused 56 (delta 8), pack-reused 186
Receiving objects: 100% (252/252), 34.89 KiB | 3.49 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [3]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1QngGsa957_hUttMrc6hGaww5LTjVdQQF' -O interval_newton
!chmod +x ./interval_newton

--2023-12-11 00:19:33--  https://docs.google.com/uc?export=download&id=1QngGsa957_hUttMrc6hGaww5LTjVdQQF
Resolving docs.google.com (docs.google.com)... 172.217.15.238, 2607:f8b0:4025:815::200e
Connecting to docs.google.com (docs.google.com)|172.217.15.238|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lj76o2u68l4au8akca88tvqrfh4iri0m/1702253925000/02668767947819874432/*/1QngGsa957_hUttMrc6hGaww5LTjVdQQF?e=download&uuid=6684e96b-fa61-445a-b456-e9d9aeba27ec [following]
--2023-12-11 00:19:35--  https://doc-10-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/lj76o2u68l4au8akca88tvqrfh4iri0m/1702253925000/02668767947819874432/*/1QngGsa957_hUttMrc6hGaww5LTjVdQQF?e=download&uuid=6684e96b-fa61-445a-b456-e9d9aeba27ec
Resolving doc-10-3c-docs.googleusercontent.com (doc-10-3c-docs.googleusercontent.com)... 172.217.0.65, 2607:f8b0:4025:810::200

In [4]:
!gdown https://drive.google.com/uc?id=1lS5q0100EgUZYMjetYPy7jf4TG6o_JgR
!chmod +x ibexopt

Downloading...
From: https://drive.google.com/uc?id=1lS5q0100EgUZYMjetYPy7jf4TG6o_JgR
To: /content/ibexopt
100% 4.46M/4.46M [00:00<00:00, 28.2MB/s]


In [5]:
import numpy as np
import subprocess
import re
import os
import matplotlib.pyplot as plt
from sympy import symbols, Matrix, diff, lambdify, sympify, cos, sin, pi, exp, sqrt #importar las necesarias
from sklearn.linear_model import LinearRegression
from sympy.core.sympify import SympifyError
from scipy.optimize import minimize
from os import walk # Para leer el nombre de los archivos en la carpeta de tft_problems
import time # Para calcular tiempos

#GENERAL

## Funciones txt y regresión

In [6]:
"""
Función encargada de leer el contenido del problema txt y retornar
la funcion objetivo, variables y dominios respectivos
"""
def open_text(directory):
# Lee el contenido del archivo
  with open(directory, 'r') as f:
      content = f.read()

  # Extrae las variables y sus límites
  variable_lines = re.findall(r'(\w+)\s+in\s+\[(-?[\d.]+),(-?[\d.]+)\];', content)
  x_str = [var_info[0] for var_info in variable_lines]
  x_bounds = [[float(var_info[1]), float(var_info[2])] for var_info in variable_lines]


  # Extrae la función objetivo
  function_txt = re.search(r'minimize\s+([\s\S]+?);', content).group(1).strip()
  function = function_txt.replace('^', '**')

  #Convertimos a symbols la funcion objetivo y las variables
  obj_function, x_vars = convert_symbols(function, x_str)

  return x_vars, x_bounds, obj_function, function

"""
Función encargada de encontrar una solucion al problema, acotandole sus bounds con beta y ajustandola
a g(x)=0 en caso de ser necesario, generando así uina nueva funcion.

"""
def get_bounds_and_function(max_iter, obj_function, x_vars, beta, activate_constraint, x_bounds):
  random_point, value_function = get_negative_point(max_iter, obj_function, x_vars, x_bounds)

  new_bounds, new_function = reduce_bounds_from_value(x_bounds, beta, random_point, value_function, obj_function, activate_constraint )

  return new_bounds, new_function

"""
Función encargada de generar la regresión lineal para obtener el valor de a. De
igual manera, con el valor de a se encarga de generar la nueva funcion para la propuesta y
realizar la acotación de los bounds a un rango +- beta
"""
def get_sampling(new_bounds, regresion_points):
  sampling = np.array([np.random.uniform(low, high, size=regresion_points) for low, high in new_bounds])

  return sampling

"""
Función encargada de almacenar las variables de aritmetica afín, KKT conditions,
y SLSQP en un txt donde posteriormente será visualizado para su análisis
"""
def save_result_txt(problem_number: int, beta_value: float, lambda_interval: list, active_constr: bool,
                    aff_a: list, aff_hc4: list, aff_newton: list, aff_newton_err: float, aff_dif: float, aff_x: list, aff_ev: float, aff_hessian: str, aff_error: list,
                    slsqp_x: list, slsqp_ev: float, ibex_lb_ub, ibex_it, problem_iter):
  path = "results"
  if not os.path.exists(path):
    os.mkdir(path)
  problem_path = "results/problem_"+str(problem_number)
  if not os.path.exists(problem_path):
    os.mkdir(problem_path)

  aff_func = prop_obj_function(0, aff_a , new_function, x_vars, False, lambda_interval)
  txt_name = str(problem_iter)+"_lambda"
  if lambda_interval == None:
    txt_name = str(problem_iter)+"_lambda0"
  try:
    # Abre un archivo en modo escritura
    with open(problem_path+"/"+txt_name+".txt", "w") as file:
        # Escribe los parámetros en el archivo
        file.write(f"Beta Value: {beta_value}\n")
        file.write(f"Lambda bounds: {lambda_interval}\n")
        file.write(f"Restricción activa?: {active_constr}\n")
        file.write("----------------------------- KKT (PENDIENTE DE A.A.) --------------------------------------\n")
        file.write(f"Función de AA: {aff_func}\n")
        file.write(f"Pendiente de AA: {aff_a}\n")
        file.write(f"HC4 de AA: {aff_hc4}\n")
        file.write(f"Newton de AA: {aff_newton}\n")
        file.write(f"X de AA: {aff_x}\n\n")
        file.write(f"Precisión de newton en AA: {aff_newton_err}\n")
        file.write(f"Diferencia hc4 y newton (AA): {aff_dif}\n")
        file.write(f"Ev de AA: {aff_ev}\n")
        file.write(f"Hessiano de AA: {aff_hessian}\n")
        file.write(f"Intervalo de error de AA: {aff_error}\n")
        file.write('------------------------------ SLSQP -------------------------------------\n')
        file.write(f"X de slsqp: {slsqp_x}\n")
        file.write(f"Ev de slsqp: {slsqp_ev}\n")
        file.write('------------------------------ IBEX -------------------------------------\n')
        file.write(f"[lb, ub]: [{float(ibex_lb_ub[0])}, {float(ibex_lb_ub[1])}]\n")
        file.write(f"nodes: {ibex_it}\n")

    print(f"Los parámetros han sido guardados en '{txt_name}.txt' correctamente.")
  except Exception as e:
    print(f"Error al guardar los parámetros: {e}")
  return

def save_result_summary(problem_number, c_kkt, sqp, AA_error, noconv_count, saddle_count, iter, AA_mean_time, KKT_mean_time):
  problem_path = "results/problem_"+str(problem_number)
  txt_name = "problem_"+str(problem_number)+"_summary.txt"
  txt_path = problem_path+"/"+txt_name

  sum = 0
  success_percent = 100*(1-(noconv_count+saddle_count)/len(c_kkt))

  c_kkt = np.array(c_kkt)
  sqp = np.array(sqp)

  AA_error = np.array(AA_error)
  aa_lb_mean = np.mean(AA_error[:, 0])
  aa_ub_mean = np.mean(AA_error[:, 1])

  # Se obtiene el menor valor de cada una de las iteraciones y se calcula su promedio
  for item in c_kkt:
      if item[0] is not None and (item[1] is None or item[0] < item[1]):
          sum += item[0]
      elif item[1] is not None and (item[0] is None or item[1] < item[0]):
          sum += item[1]

  mean_kkt = sum / len(c_kkt)

  sqp = sqp[sqp != None]
  mean_sqp = sqp.mean()
  try:
  # Abre un archivo en modo escritura
    with open(txt_path, "w") as file:
        # Escribe los parámetros en el archivo
        file.write(f"Iteraciones : {iter}\n")
        file.write(f"Promedio KKT : {mean_kkt}\n")
        file.write(f"Promedio SLSQP: {mean_sqp}\n")
        file.write(f"Promedio AA : [{aa_lb_mean}, {aa_ub_mean}]\n")
        file.write(f"noconv_count : {noconv_count}\n")
        file.write(f"saddle_count : {saddle_count}\n")
        file.write(f"porcentaje de éxito: {success_percent}\n")
        file.write(f"Tiempo medio AA : {AA_mean_time}\n")
        file.write(f"Tiempo medio KKT: {KKT_mean_time}\n")
    print(f"Los parámetros han sido guardados en '{txt_path}' correctamente.")
  except Exception as e:
    print(f"Error al guardar el resumen del problema '{problem_number}': {e}")

  return

##Funciones KKT

In [7]:
"""
Función encargada de realizar una conversión a simbolos de las
variables de decisión y la funcion.
"""
def convert_symbols(f, x):
    x_sympy = symbols(x)
    f_sympy = sympify(f)
    return f_sympy.subs(zip(x, x_sympy)), x_sympy


"""
Función encargada de definir la función objetivo del problema de
optimización para la regresión lineal
"""
def regr_obj_function(x_values, f, x_symbols, slsqp_condition):


    # Crear símbolos para los valores de las variables
    x_vals_symbols = symbols('x_vals:' + str(len(x_symbols)))
    subs_dict = {x: x_val_symbol for x, x_val_symbol in zip(x_symbols, x_vals_symbols)}

    # Sustituir los símbolos por los símbolos de valores reales
    f_subs = f.subs(subs_dict)
    # Crear una función NumPy a partir de la expresión simbólica
    f_lambda = lambdify(x_vals_symbols, f_subs, "numpy")

    if (slsqp_condition):
      result = f_lambda(*x_values)
    else:
      # Evaluar la función NumPy en los valores dados
      result = np.array([f_lambda(*x_val) for x_val in zip(*x_values)])

    return result

"""
Función encargada de definir la función objetivo del problema de
optimización con su linearización
"""
def prop_obj_function(x_values, a, f, x_symbols, slsqp_condition, lambda_bounds):

  if type(x_values) != int:
    if len(a) < len(x_values) and len(x_values) > 0:
      x_values = x_values[:-1]

  if (slsqp_condition):

    sum = np.dot(a, x_values)

    # Crear símbolos para los valores de las variables
    x_vals_symbols = symbols('x_vals:' + str(len(x_symbols)))
    subs_dict = {x: x_val_symbol for x, x_val_symbol in zip(x_symbols, x_vals_symbols)}

    # Sustituir los símbolos por los símbolos de valores reales
    f_subs = f.subs(subs_dict)
    # Crear una función NumPy a partir de la expresión simbólica
    f_lambda = lambdify(x_vals_symbols, f_subs, "numpy")

    result = f_lambda(*x_values) - (sum)

  else:
    sum = np.dot(a, x_symbols)
    result = f - (sum)

  return result

"""
Función encargada de definir la restricción del problema de
optimización.
"""
def constr(f):
  return f

"""
 Función para reducir los bounds de las variables a
 partir del valor actual de cada variable
"""
def reduce_bounds_from_value(bounds, beta, point, value_function, function, activate_constraint):

  new_function = function

  # Modificamos la funcion si queremos activar g(x)' = 0
  if (activate_constraint):
    new_function = function - value_function

  #Reducimos dominios considerando el punto y el parámetro beta
  reduced_bounds = []

  for i, (low, high) in enumerate(bounds):
      new_low = max(low, point[i] - beta)
      new_high = min(high, point[i] + beta)
      reduced_bounds.append([new_low, new_high])

  return reduced_bounds, new_function

"""
Función encargada de definir la función objetivo y restricción en
su versión Lagrangiana.
"""
def lagrange_convert(a, f, x, lambda_bounds):

    if lambda_bounds != None:
      # Definimos la variable simbólica para lambda
      λ = symbols('λ')

      # Definimos la función Lagrange
      L = prop_obj_function(0, a, f, x, False, lambda_bounds) + λ * constr(f)
    else:
      L = prop_obj_function(0, a, f, x, False, lambda_bounds)

    return L

"""
Función encargada de calcular la derivada con respecto a x de la
versión Lagrangiana del problema.
"""
def deriv_calculate(L, x):
  dL = []

  # Calculamos la derivada parcial de cada una de las variables
  for var in x:
    dL.append(diff(L, var))

  return dL

"""
Función encargada de reemplazar '**' con '^' y los 'λ' por 'y'
"""
def convert_to_txt(eq):
    eq = eq.replace('**', '^')
    eq = eq.replace('λ', 'l')
    return eq

"""
Función encargada de generar el sistema de ecuaciones para la derivada
parcial de Lagrange
"""
def equation_generator(dL, f, lambda_bounds):
    eqs = []

    # Generamos el sistema de ecuaciones para cada derivada parcial
    for dL_var in dL:
      eqs.append(convert_to_txt(f"{dL_var} = 0"))

    if lambda_bounds != None:
      λ = symbols('λ')
      #Condición de complementariedad
      eqs.append(convert_to_txt(f"{λ*constr(f)} = 0"))

    return eqs

"""
Función encargada de guardar las variables con sus respectivos bounds y
el sistema de ecuaciones asociado para la ejecución del método
de Intervalos de Newton
"""
def save_txt(eqs, x_bounds, lambda_bounds):
  with open('test.txt', 'w') as file:
      # Escribir la sección Variables
      file.write("Variables\n\n")
      for i, x_bound in enumerate(x_bounds, start=1):
          file.write(f"x{i} in [{x_bound[0]},{x_bound[1]}];\n")

      if lambda_bounds != None:
        file.write(f"l in [{lambda_bounds[0]},{lambda_bounds[1]}];\n")

      # Escribir la sección Constraints
      file.write("\nConstraints\n")
      for eq in eqs:
          file.write(f"{eq};\n")
      file.write("end")

"""
Función encargada de generar una solucion aleatoria para la funcion objetivo inicial
con el fin de poder acotar los bounds del problema
"""
def get_negative_point(max_iter, obj_function, x_vars, x_bounds):
  evaluation = 1
  random_point = None

  for i in range(max_iter):
    if (evaluation <= 0):
      break
    random_point = np.array([np.random.uniform(low, high, size=1) for low, high in x_bounds ] ) #[[-10,10]]*len(x_vars) ] )
    evaluation = regr_obj_function(random_point, obj_function, x_vars, False)

  if random_point is not None:
    random_point = np.squeeze(random_point)

  return random_point, evaluation[0]

"""
Función encargada de generar las condiciones KKT del problema y además
guardar el archivo txt para el metodo de newto interval
"""
def generate_kkt(a, new_function, x_vars, new_bounds, lambda_bounds):

  # Calculamos lagrange de la funcion y restricción
  lagrange = lagrange_convert(a, new_function, x_vars, lambda_bounds)

  # Calculamos la derivada con respecto a x de lagrange
  dL = deriv_calculate(lagrange, x_vars)

  # Obtenemos el sistema de ecuaciones en el vector eqs
  eqs = equation_generator(dL, new_function, lambda_bounds)

  #Guardamos el txt para el metodo de Newton
  save_txt(eqs, new_bounds, lambda_bounds)

  return lagrange, dL, eqs

import numpy as np

def calculate_difference(hc4, newton):
    if len(hc4) != len(newton):
        raise ValueError("HC4 y newton tienen distinta longitud.")

    diferencias = []

    for arr1, arr2 in zip(hc4, newton):
        diferencia = np.subtract(arr1, arr2)
        diferencia = np.abs(diferencia)
        suma = diferencia.sum()
        diferencias.append(suma)

    diferencias = np.array(diferencias)
    return diferencias.sum()

"""
Función encargada de calcular de el resultado optimo del problema
a través del intervalo de newton

"""
def get_newton_result(new_function, x_vars, max_newton_iter, new_bounds, a_AA, lambda_bounds):
  newton_empty = False

  lagrange, dL, eqs = generate_kkt(a_AA, new_function, x_vars, new_bounds, lambda_bounds)

  data = !./interval_newton test.txt -c newton

  if data[0] == 'empty vector':
    newton_empty = True

  return newton_empty, data

Funciones auxilaries de parseo

In [8]:
def convert_to_vars_string(x_vars):
    n_vars = len(x_vars)
    vars_string = 'x[{}]'.format(n_vars)
    return vars_string

def convert_to_function_string(new_function):
    def replace(match):
        return f'x[{int(match.group(1)) - 1}]'

    pattern = r'x(\d+)'
    converted_expr = re.sub(pattern, replace, new_function)
    converted_expr = converted_expr.replace('**', '^')

    return converted_expr


#ARITMÉTICA AFÍN

## Generador AF

In [9]:
from pyibex import Interval, IntervalVector
import math

class AF:
    def __init__(self, a, x, error=Interval(0,0)):
        self.a = a
        self.x = x
        self.error = error

    def __neg__(self):
        return AF([-ai for ai in self.a], self.x, -self.error)

    def __sub__(self, other):

      return self + (-other)
        # if other is None:
        #     return self
        # new_a = [ai - bi for ai, bi in zip(self.a, other.a)]
        # new_error = self.error - other.error

        # return AF(new_a, self.x, new_error)
    def __truediv__(self, other):
            return self * other**(-1)

    def __pow__(self, exponent):
        if isinstance(exponent, int):
          if exponent==2:
            return self.cheby_convex(f=lambda x: x**2, df_proj=lambda m: m/2)
          elif exponent==-1:
            return self.cheby_convex(f=lambda x: 1/x, df_proj=lambda m: math.sqrt(-1/m))
        else:
            raise ValueError(f"exponent={exponent} is not implemented")

    def cheby_convex(af, f=lambda x: x**2, df_proj=lambda m: m/2):
      x = af.to_interval()
      m = (f(x.ub())-f(x.lb()))/x.diam() # y = f(a) + m(x-a) => mx + A
      xp = df_proj(m)     # y = f(xp) + m(x-xp) => mx + B
      A = f(x.lb())-m*x.lb()
      B = f(xp)-m*xp

      return af*m + Interval(min(A,B), max(A,B))

    def __add__(self, other):
        if isinstance(other, (int, float, Interval)):
            return AF(self.a, self.x, self.error+other)
        new_a = [ai + bi for ai, bi in zip(self.a, other.a)]
        new_error = self.error + other.error
        return AF(new_a, self.x, new_error)

    def __mul__(self, other):

        # Excepción con other None
        if other is None:
            return self

        # Caso para multiplicar con un escalar
        if isinstance(other, (int, float)):
            return AF([ai * other for ai in self.a], self.x, self.error * other)

        y = self.x - self.x.mid()

        other_y = other.x - other.x.mid()

        a0, b0 = self.to_interval().mid(), other.to_interval().mid()

        new_a = [a0 * bi + ai * b0 for ai, bi in zip(self.a, other.a)]

        pairwise_error = sum(self.a[i] * other.a[j] * y[i] * y[j]
                            for i in range(len(self.a))
                            for j in range(len(other.a)) if i != j)

        error_from_mid = (self.error - self.error.mid()) * other.to_interval() + (other.error - other.error.mid()) * self.to_interval()
        result = sum(ai * xi.mid() for ai, xi in zip(new_a, self.x))
        new_error = error_from_mid + pairwise_error + a0*b0 - result

        # Ajustando el error para coeficientes no cero
        for i, (ai, bi) in enumerate(zip(self.a, other.a)):
            if ai and bi:
                new_error += y[i]**2 * (ai * bi)

        return AF(new_a, self.x, new_error)

    def __rmul__(self, other):

        # Excepción con other None
        if other is None:
            return self

        # Caso para multiplicar con un escalar
        if isinstance(other, (int, float)):
            return AF([ai * other for ai in self.a], self.x, self.error * other)
        y = self.x - self.x.mid()

        other_y = other.x - other.x.mid()
        a0, b0 = self.to_interval().mid(), other.to_interval().mid()

        new_a = [a0 * bi + ai * b0 for ai, bi in zip(self.a, other.a)]

        pairwise_error = sum(self.a[i] * other.a[j] * y[i] * y[j]
                            for i in range(len(self.a))
                            for j in range(len(other.a)) if i != j)

        error_from_mid = (self.error - self.error.mid()) * other.to_interval() + (other.error - other.error.mid()) * self.to_interval()
        result = sum(ai * xi.mid() for ai, xi in zip(new_a, self.x))
        new_error = error_from_mid + pairwise_error + a0*b0 - result

        # Ajustando el error para coeficientes no cero
        for i, (ai, bi) in enumerate(zip(self.a, other.a)):
            if ai and bi:
                new_error += y[i]**2 * (ai * bi)

        return AF(new_a, self.x, new_error)
    def to_interval(self):
        result = Interval(0)
        for ai, xi in zip(self.a, self.x):
            result += ai * xi
        result += self.error
        return result

    @staticmethod
    def from_intervalvector(x):
        afs = [] #Editado por mí
        n=x.size()

        for i, x_i in enumerate(x):
            a = [0] * n
            a[i] = 1
            afs.append(AF(a, x))
        return afs

    def __str__(self):
      s = ""
      mid_x = self.x.mid()
      for i, ai in enumerate(self.a):
          if ai == 0: continue
          if s!="": s+= " + "
          if ai == 1.0: s += f"x_{i}"
          else: s += f"{ai}*x_{i}"
      s += f" + {str(self.error)}"
      return s

def exp(af):
    return af.cheby_convex(f=lambda x: math.exp(x), df_proj=lambda m: math.log(m))

def sqrt(af):
    return af.cheby_convex(f=lambda x: math.sqrt(x), df_proj=lambda m: 1/(4*m**2))


## Funciones AF

In [10]:
"""
Función encargada de reemplazar las variables x1, x2, x3... en la función obj por el
arreglo afs_x que contiene los objetos AF. Retornando así la función en formato str.
"""
def reemplazar_variables(obj_function, x_vars, afs_x):

  for i in range(1, len(x_vars) + 1):
    x_var = "x" + str(i)
    x = "x[" + str(i-1) + "]"
    obj_function = obj_function.replace(x_var, x)

  return obj_function

"""
Se asocia cada objeto AF a las variables del problema con el fin de evaluar su contenido
con la funcion objetivo.
"""
def create_dictionary_afs(x_vars, afs_x):
  afs_values = {}
  for i in range(0, len(x_vars)):
      x_var = "x[" + str(i) + "]"
      afs_values[x_var] = afs_x[i]

  return afs_values

"""
Función encargada de realizar la linealización Afín Arithmetic y retornar dicho
resultado con respectivo formato.
"""
def aa_linealization(function, x_vars, new_bounds):

  x = IntervalVector([Interval(bound[0], bound[1]) for bound in new_bounds ])

  afs_x = AF.from_intervalvector(x)

  # Reemplazar las variables
  obj_function_afs = reemplazar_variables(function, x_vars, afs_x)

  # Crear un diccionario que asocie las variables en afs_function con los valores de afs_x
  afs_values = create_dictionary_afs(x_vars, afs_x)

  result = eval(obj_function_afs, {"x":afs_x})

  return result

## Funciones SLSQP

In [11]:
"""
Función encargada de generar un punto aleatorio en los rangos definidos por los
bounds del intervalo. Este punto aleatorio es el punto inicial para el algoritmo
SLSQP

"""
def random_initial_point(bounds):
  initial_point = []
  for i in range(len(bounds)):
    variable_bound = bounds[i]
    n = np.random.uniform(variable_bound[0], variable_bound[1])
    initial_point.append(n)
  return initial_point

##Funciones Hessian

In [12]:
"""
Función encargada de verificar los valores propios de la matriz hessiana
y retornar el tipo de punto
"""
def hess_check(hessian_matrix):
  # Valores propios de la matriz hessiana
  vp = np.linalg.eigvals(hessian_matrix)

  if np.all(vp > 0):
    minimum = True
    return "Minimum"

  elif np.all(vp < 0):
    maximum = True
    return "Maximum"

  elif np.any(vp < 0) and np.any(vp > 0):
    saddle = True
    return "Saddle"

  elif np.any(vp == 0):
    not_conclusive = True
    return "Not Conclusive"


"""
Función encargada de convertir la salida de HC4 a un formato legible para su utilización
"""
def convert_formated_X(entrada):
  # Utilizamos expresiones regulares para encontrar intervalos con corchetes o signos de mayor/menor
  intervalos = re.findall(r'[<\[]([^<>\[\]]+)[>\]]', entrada)

  resultado = []

  for intervalo in intervalos:
      numeros = intervalo.split(',')
      numeros = [float(num.strip()) if '.' in num or 'e' in num else int(num.strip()) for num in numeros]
      resultado.append(numeros)

  return resultado


"""
Función encargada de calcular el error asociado a los intervalos de solución
generada por newton con el fin de saber que tanto contrató hacia la solución

"""
def calculate_error_X(X):
  array_error = []
  point = []

  for interval in X:
    error = interval[0] - interval[1]
    array_error.append(error)

    point_var = np.mean(interval)
    point.append(point_var)

  array_error = np.abs(array_error)

  return array_error, np.mean(array_error), point

"""
Función encargada de destructurar la salida desde el metodo de interval newton
a un formato legible e utilizable.

"""
def get_solution_newton(data):
  # Extraimos HC4 desde data (salida de newton)
  HC4_str = data[0]
  X_str = data[-1]
  # Realizamos el formateo de X y HC4 desde str a array
  X_formated = convert_formated_X(X_str)
  HC4_formated = convert_formated_X(HC4_str)

  # Extraimos la matriz de soluciones de las variables
  matrix_str = data[2:-1]

  # Extrae los valores de la matriz Hessiana de cada línea
  hessian_values = []
  for line in matrix_str:
      values = re.findall(r"-?\d+(?:\.\d+)?(?:e[+-]?\d+)?", line)
      hessian_values.extend([float(val) for val in values])

  # Calcula la dimensión de la matriz Hessiana
  hessian_dim = int(len(hessian_values) ** 0.5)

  # Crea el array de NumPy
  hessian_matrix = np.array(hessian_values).reshape(hessian_dim, hessian_dim)

  # Calculamos el error de X
  error_x, mean_error, point_x = calculate_error_X(X_formated)

  return  hessian_matrix, HC4_formated, X_formated, error_x, mean_error, point_x

## Funciones ibex


In [13]:
def bounds(vars, xl, x, ctrs=None, prec=1e-8):
  lb = None  # Inicializa lb con un valor por defecto
  ub = None  # Inicializa ub con un valor por defecto
  n_cells = 0  # Inicializa n_cells con un valor por defecto

  with open('tmp_min.txt', 'w') as f:

    bounds = str(x)
    bounds = re.sub(r'\], \[', ']; [', bounds)
    bounds = bounds.replace("[[", "[").replace("]]", "]")

    f.write('variables\n')
    f.write(vars+' in '+ '(' + str(bounds)+ ')'+';\n\n')
    f.write('minimize '+xl+';')
    if ctrs is not None:
      f.write('\nconstraints\n')
      for ctr in ctrs: f.write(ctr+'\n')
      f.write("end\n")

  #Minimización
  p_min = subprocess.Popen(["./ibexopt", '-a', '1e-8', '-r', str(prec), '-t', '5', 'tmp_min.txt'], stdout=subprocess.PIPE)

  for line in p_min.communicate()[0].decode().splitlines():
    #print(line)
    if line.find("f* in")!=-1:
      lb = float(re.search('\[(.*),(.*)\]', line).groups(0)[1])

    if line.find("number of cells:")!=-1:
      n_cells = int(re.search('\t\t([0-9]*)', line).groups(0)[0])


  #Maximización
  with open('tmp_max.txt', 'w') as f:

    bounds = str(x)
    bounds = re.sub(r'\], \[', ']; [', bounds)
    bounds = bounds.replace("[[", "[").replace("]]", "]")

    f.write('variables\n')
    f.write(vars+' in '+ '(' + str(bounds)+ ')'+';\n\n')
    f.write('minimize -('+xl+');\n')
    if ctrs is not None:
      f.write('\nconstraints\n')
      for ctr in ctrs: f.write(ctr+'\n')
      f.write("end\n")

  p_max = subprocess.Popen(["./ibexopt", '-a', "1e-8", '-r', str(prec), '-t', '5', 'tmp_max.txt'], stdout=subprocess.PIPE)

  for line in p_max.communicate()[0].decode().splitlines():
    #print(line)
    if line.find("f* in")!=-1:
      ub = -float(re.search('\[(.*),(.*)\]', line).groups(0)[0])

    if line.find("number of cells:")!=-1:
      n_cells += int(re.search('\t\t([0-9]*)', line).groups(0)[0])

  return (lb,ub), n_cells

In [14]:
"""
Función encargada de realizar el parseo de las valores asociados a x_vars
a formato ibex

"""
def convert_to_vars_string(x_vars):
    n_vars = len(x_vars)
    vars_string = 'x[{}]'.format(n_vars)
    return vars_string

"""
Función encargada de realizar el parseo de la funcion objetivo y las restricciones
a formato ibex

"""
def convert_to_function_string(function):
    def replace(match):
        return f'x[{int(match.group(1)) - 1}]'

    pattern = r'x(\d+)'
    converted_expr = re.sub(pattern, replace, function)
    converted_expr = converted_expr.replace('**', '^')

    return converted_expr


#GENERADOR KKT

## Main KKT

In [15]:
def calculate_relative_distances(kkt_c, ev_SLSQP, error_AA, ibex_lb, ibex_ub, hessian):
  kkt, sqp = None, None

  a, b = error_AA
  w = ibex_ub - ibex_lb

  aff_lb = abs((ibex_lb-a)/w)
  aff_ub = abs((b - ibex_ub)/w)

  if hessian == 'Minimum':
    print("minimo\n")
    kkt = abs((ibex_lb - kkt_c)/w)
    sqp = abs((ibex_lb - ev_SLSQP)/w)

  if hessian == 'Maximum':
    print("máximo\n")
    kkt = abs((kkt_c - ibex_ub)/w)
    sqp = abs((ev_SLSQP - ibex_ub)/w)


  return kkt, sqp, (aff_lb, aff_ub)

In [19]:
# mypath = '/content/tft_problems/'
# filenames = next(walk(mypath), (None, None, []))[2]  # [] if no file
# filenames.remove("README.md")
# for problem_number in filenames:
# Parámetros
lambda_bounds_values = [None, [0.001, 10e5]]    # Utilizar con None si si lambda=0, sino como arreglo con dos valores tipo [lb, ub] ej: [0.001, 10e5]
beta = 5                              # Parametro de acotación de los bounds
activate_constraint = True                      # True = se activa g(x) con g(x) = 0, False = solo se consideran los puntos que cumplan g(x)
max_iter = 1000                                 # Cantidad de iteraciones que utiliza el algoritmo para generar el punto aleatorio inicial al evaluar g(x)
max_newton_iter = 10                            # Cantidad de iteraciones máximas para que newton no tire empty vector
regresion_points = 1000                         # Cantidad de puntos para el sampling de la regresión
problem_number = "problem_13.txt"                          # Número del problema
prec=1e-8                                       # Precisión de ibex
iter = 100                                      # Iteraciones para las pruebas
tolerancia = 0.01                               # Porcentaje de tolerancia al comparar kkt_c y el mínimo y máximo de ibex
iter_while = iter


noconv_count = 0
saddle_count = 0
newton_oob = False
sqp_res = []
aff_res = []
dist_kkt = []
dist_total_kkt_c = []
AA_cputimes = []
KKT_cputimes = []



print(f"Procesando el problema {problem_number}...\n")
problem_directory = f'/content/tft_problems/{problem_number}'

# Almacenamiento de las variables del problem.txt
x_vars, x_bounds, obj_function, function_str = open_text(problem_directory)

while iter_while > 0:
  dist_c = []

  # Calculo de la funcion ajustada y sus respectivos bounds
  new_bounds, new_function = get_bounds_and_function( max_iter, obj_function, x_vars, beta, activate_constraint, x_bounds )

  # #--------------------------------------- Implementacion de AF -------------------------------------------------------#
  AA_start_time = float(time.process_time())
  new_function_str = str(new_function)
  x_AA = aa_linealization(new_function_str, x_vars, new_bounds)
  a_AA = np.array(x_AA.a, dtype = np.float64)
  error_AA = x_AA.error
  AA_finish_time= float(time.process_time())
  #--------------------------------------- IBEX -------------------------------------------------------#
  # reestructurar las variables del problema (x_vars, new_function, new_bounds, constrains) con funciones auxiliares
  prop_function = prop_obj_function(0, a_AA, new_function, x_vars, False, None)

  # Conversión x_vars a string (formato ibex)
  vars_string = convert_to_vars_string(x_vars)

  # Conversión de la funcion obj a formato ibex
  function_str = str(prop_function)
  function_string = convert_to_function_string(function_str)

  # Agregamos la constraint
  constraint = constr(str(new_function))
  constraint_string = convert_to_function_string(constraint)
  ctrs = []
  ctrs.append(constraint_string+"<=0"+";")

  y_lb_ub, ibex_it = bounds(vars_string, function_string, new_bounds, ctrs, prec)
  ibex_lb, ibex_ub = float(y_lb_ub[0]), float(y_lb_ub[1])

  #--------------------------------------- KKT -------------------------------------------------------#
  for lambda_bounds in lambda_bounds_values:
    KKT_start_time = float(time.process_time())
    print("----------------------------------------------")
    print("Calculando KKT para lambda: ", lambda_bounds)
    # Se obtiene el valor de Newton con las pendientes de AF
    empty_vector, data = get_newton_result(new_function, x_vars, max_newton_iter, new_bounds, a_AA, lambda_bounds)
    KKT_finish_time = float(time.process_time())
    if (empty_vector):
      print('--- ERROR: Empty vector en KKT conditions ---')
      dist_c.append(None)  #<---------------------------
    else:
      # Variables obtenidades desde Newton
      hessian_matrix_AA, HC4_AA, X_AA, error_X_AA, mean_error_AA, point_x_AA =  get_solution_newton(data)

      if len(X_AA) == 0:
        print('--- ERROR: Se generó DATA pero tiene un "EMPTY VECTOR" ---')
        dist_c.append(None)  #<---------------------------

      else:

        # Se comprueba mid(x) esté en los bounds acordes de las variables
        i=0
        for lb, ub in new_bounds:
          if point_x_AA[i] < lb or point_x_AA[i] > ub:
            print(f"Error: al menos una variable fuera de bounds en newton")
            dist_c.append(None)  #<---------------------------
            newton_oob = True
            break
          i=i+1

        if (newton_oob):
          continue

        # Se obtienen los resultados de las variables de manera independiente a almacenar en el txt

        kkt_c = prop_obj_function(point_x_AA, a_AA, new_function, x_vars, True, lambda_bounds)

        # Comprobar que el valor de "c" se encuentre dentro del intervalo de ibex
        new_ibex_lb = ibex_lb*(1+tolerancia)
        new_ibex_ub = ibex_ub*(1+tolerancia)
        tol_numeric_lb = abs(ibex_lb*tolerancia)
        tol_numeric_ub = abs(ibex_ub*tolerancia)

        if not math.isclose(kkt_c, ibex_lb, rel_tol = tol_numeric_lb) and not math.isclose(kkt_c, ibex_ub, rel_tol = tol_numeric_ub):
          print("Valor de kkt: ",kkt_c)
          print(f"Intervalo ibex: [{ibex_lb}, {ibex_ub}]")
          print(f"Intervalo modificado: [{new_ibex_lb}, {new_ibex_ub}]")
          print(f"Error: evaluación de kkt no se encuentra en intervalo de ibex")
          dist_c.append(None)  #<---------------------------
          continue

        aa_dif = calculate_difference(HC4_AA, X_AA)
        aa_hessian = hess_check(hessian_matrix_AA)

    #--------------------------------------- Implementacion de SLSQP -------------------------------------------------------#
    sampling = get_sampling(new_bounds, regresion_points)
    initial_point = random_initial_point(new_bounds)
    regr_obj_function(sampling, new_function, x_vars, False)
    constraint = {'type': 'ineq', 'fun': regr_obj_function, 'args': (new_function, x_vars, True)}
    slsqp = minimize(prop_obj_function, initial_point, args=(a_AA, new_function, x_vars, True,  lambda_bounds), method='SLSQP', bounds= new_bounds, constraints= constraint)
    x_SLSQP = slsqp.x
    ev_SLSQP = slsqp.fun

    #--------------------------------------- TXT de salida -------------------------------------------------------#
    if empty_vector == False and (len(X_AA) > 0):
      iter_while = iter_while -1
      current_iter = iter - iter_while

      print("Iteración", current_iter)

      save_result_txt(problem_number, beta, lambda_bounds, activate_constraint,
                a_AA, HC4_AA, X_AA, mean_error_AA,  aa_dif, point_x_AA, kkt_c, aa_hessian, error_AA,
                x_SLSQP, ev_SLSQP,
                  y_lb_ub, ibex_it, current_iter)

      dist_kkt, dist_sqp, dist_aff = calculate_relative_distances(kkt_c, ev_SLSQP, error_AA, ibex_lb, ibex_ub, aa_hessian)
      sqp_res.append(dist_sqp)
      aff_res.append(dist_aff)

      dist_c.append(dist_kkt) #<---------------------------

      # calcular tiempos
      AA_cputimes.append(AA_finish_time - AA_start_time)
      KKT_cputimes.append(KKT_finish_time - KKT_start_time)
      # Para contar cuando no converge
      if mean_error_AA > 1e-7:
        noconv_count = noconv_count + 1
        continue
      if (aa_hessian == 'Saddle' or aa_hessian == 'Not Conclusive'):
        saddle_count = saddle_count + 1
        continue

  dist_total_kkt_c.append(dist_c) #<---------------------------
np_AA_cputimes = np.array(AA_cputimes)
np_KKT_cputimes = np.array(KKT_cputimes)
save_result_summary(problem_number, dist_total_kkt_c, sqp_res, aff_res, noconv_count, saddle_count, iter, np_AA_cputimes.mean(), np_KKT_cputimes.mean())

Procesando el problema problem_13.txt...

----------------------------------------------
Calculando KKT para lambda:  None
Iteración 1
Los parámetros han sido guardados en '1_lambda0.txt' correctamente.
máximo

----------------------------------------------
Calculando KKT para lambda:  [0.001, 1000000.0]
Iteración 2
Los parámetros han sido guardados en '2_lambda.txt' correctamente.
máximo

----------------------------------------------
Calculando KKT para lambda:  None
Iteración 3
Los parámetros han sido guardados en '3_lambda0.txt' correctamente.
máximo

----------------------------------------------
Calculando KKT para lambda:  [0.001, 1000000.0]
Iteración 4
Los parámetros han sido guardados en '4_lambda.txt' correctamente.
máximo

----------------------------------------------
Calculando KKT para lambda:  None
Iteración 5
Los parámetros han sido guardados en '5_lambda0.txt' correctamente.
máximo

----------------------------------------------
Calculando KKT para lambda:  [0.001, 100

In [17]:
print(dist_total_kkt_c)

[[None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [0.10895740283683648, 0.10895740283683648], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [0.09589463203350454, 0.09589463203350454], [None, None], [None, None], [None, None], [None, None], [None, None], [0.05263157894888521, 0.05263157894888521], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [0.1111111111130188, 0.1111111111130188], [None, None], [None, None], [None, None], [None, None], [0.08493516615669754, 0.08493516615669754], [0.04776272576821839, 0.04776272576821839], [None, None], [0.11111111111132461, 0.11111111111132461], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [None, None], [0.11029544016513232, 0.11029544016513232], [None, None], [0.09255034996987035, 0.09255034996987035], [No